In [ ]:
""" Classification Training with Flan-T5-Small on Enriched MTS Dialogue Dataset 
    https://huggingface.co/sarahahatee/t5-dialogue-classification-4"""


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets
!pip install evaluate
!pip install transformers
!pip install numpy
!pip install tensorflow
!pip install -U accelerate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.6 MB/s eta 0:00:00


In [3]:
import accelerate
from datasets import load_dataset, load_metric, concatenate_datasets, DatasetDict
from transformers import (
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import numpy as np
from sklearn.metrics import classification_report
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# Load training datasets
train_file_a = "/content/MTS-Dialog-TrainingSet-enriched-vectorized.csv"
# train_file_b = "data/MTS-Dialog-ValidationSet.csv"

# Load testing dataset
# test_file = "data/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv"
test_file = "/content/MTS-Dialog-ValidationSet.csv"

# Load training datasets from CSV files
# train_dataset_a = load_dataset('csv', data_files=train_file_a)['train']
# train_dataset_b = load_dataset('csv', data_files=train_file_b)['train']
combined_train_dataset= load_dataset('csv', data_files=train_file_a)['train']

# Load testing dataset from CSV file
test_dataset = load_dataset('csv', data_files=test_file)['train']

# Combine the two training datasets
# combined_train_dataset = concatenate_datasets([train_dataset_a, train_dataset_b])

# Create dialog_dataset dictionary
dialog_dataset = DatasetDict({
    "train": combined_train_dataset,
    "test": test_dataset
})


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
# loading flan-t5 instead of plain t5
checkpoint = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = checkpoint

# We prefix our tasks with "classify"
PREFIX_CLASSIFY = "Classify the topic of this dialogue: "

# Define the preprocessing function
def preprocess_function_classify(examples, include_umls_cui_mappings=False):
    """Add prefix to the sentences, tokenize the text, and set the labels"""
    # Conditionally include 'umls_cui_mappings' in the input
    if include_umls_cui_mappings and 'umls_cui_mappings' in examples:
        inputs = [PREFIX_CLASSIFY + doc + " UMLS CUI mappings: " + cui_mappings
                  for doc, cui_mappings in zip(examples["dialogue"], examples["umls_cui_mappings"])]
    else:
        inputs = [PREFIX_CLASSIFY + doc for doc in examples["dialogue"]]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # The "labels" are the tokenized outputs:
    labels = tokenizer(
        text_target=examples["section_header"], max_length=50, truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Process the datasets
tokenized_train_dataset = dialog_dataset["train"].map(
    lambda examples: preprocess_function_classify(examples, include_umls_cui_mappings=True),
    batched=True
)
tokenized_test_dataset = dialog_dataset["test"].map(
    preprocess_function_classify,
    batched=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Map:   0%|          | 0/1201 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [6]:
# Global Parameters
L_RATE = 1e-4
BATCH_SIZE = 7
PER_DEVICE_EVAL_BATCH = 7
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 20

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    "t5-dialogue-classification-4",
    evaluation_strategy="epoch",
    learning_rate=L_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
    weight_decay=WEIGHT_DECAY,
    save_total_limit=SAVE_TOTAL_LIM,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    push_to_hub=True,
)


def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Convert sequences to strings or keep as-is if already strings
    decoded_preds = [
        str(pred) for pred in decoded_preds
    ]  # Convert to string if necessary
    decoded_labels = [
        str(label) for label in decoded_labels
    ]  # Convert to string if necessary

    # Generate classification report
    report = classification_report(decoded_labels, decoded_preds, digits=4)

    return {"classification_report": report}


data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

len_test_dataset = len(tokenized_test_dataset)
print(f"Length of Test Dataset: {len_test_dataset}")

Length of Test Dataset: 100


In [7]:
trainer.train()

trainer.push_to_hub()

model.to("cpu")

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Classification Report
1,No log,0.709578,precision recall f1-score support ALLERGY 0.3333 0.2500 0.2857 4 ASSESSMENT 0.0000 0.0000 0.0000 4 Afib 0.0000 0.0000 0.0000 0 CC 0.0000 0.0000 0.0000 4 DIAGNOSIS 0.0000 0.0000 0.0000 1 DISPOSITION 0.0000 0.0000 0.0000 2 EDCOURSE 0.0000 0.0000 0.0000 3 EXAM 0.0000 0.0000 0.0000 1 FAM/SOCHX 0.7222 0.5909 0.6500 22 GENHX 0.3846 1.0000 0.5556 20 GYNHX 0.0000 0.0000 0.0000 1 I T 0.0000 0.0000 0.0000 0 I V 0.0000 0.0000 0.0000 0 IMAGING 0.0000 0.0000 0.0000 1 IMMUNIZATIONS 0.0000 0.0000 0.0000 1 LABS 0.0000 0.0000 0.0000 1 MEDICATIONS 0.4444 0.5714 0.5000 7 OTHER_HISTORY 0.0000 0.0000 0.0000 1 PASTMEDICALHX 0.2000 0.2500 0.2222 4 PASTSURGICAL 0.8000 0.5000 0.6154 8 PLAN 0.0000 0.0000 0.0000 3 PROCEDURES 0.0000 0.0000 0.0000 1 Psychiatric 0.0000 0.0000 0.0000 0 ROS 0.0000 0.0000 0.0000 11 SPECIALHX 0.0000 0.0000 0.0000 0 accuracy 0.4300 100 macro avg 0.1154 0.1265 0.1132 100 weighted avg 0.3523 0.4300 0.3587 100
2,No log,0.530812,precision recall f1-score support ALLERGY 0.5000 0.2500 0.3333 4 ASSESSMENT 0.0000 0.0000 0.0000 4 Afib 0.0000 0.0000 0.0000 0 CC 0.0000 0.0000 0.0000 4 DIAGNOSIS 0.0000 0.0000 0.0000 1 DISPOSITION 0.0909 0.5000 0.1538 2 EDCOURSE 0.0000 0.0000 0.0000 3 EXAM 0.0000 0.0000 0.0000 1 FAM/SOCHX 0.8750 0.9545 0.9130 22 GENHX 0.6129 0.9500 0.7451 20 GYNHX 0.0000 0.0000 0.0000 1 How are you feeling? 0.0000 0.0000 0.0000 0 IMAGING 0.0000 0.0000 0.0000 1 IMMUNIZATIONS 0.0000 0.0000 0.0000 1 LABS 0.0000 0.0000 0.0000 1 MEDICATIONS 0.6250 0.7143 0.6667 7 OTHER_HISTORY 0.0000 0.0000 0.0000 1 PASTMEDICALHX 0.2727 0.7500 0.4000 4 PASTSURGICAL 0.7273 1.0000 0.8421 8 PLAN 0.0000 0.0000 0.0000 3 PROCEDURES 0.0000 0.0000 0.0000 1 ROS 0.0000 0.0000 0.0000 11 accuracy 0.5800 100 macro avg 0.1684 0.2327 0.1843 100 weighted avg 0.4497 0.5800 0.4963 100
3,0.608300,0.533623,precision recall f1-score support ALLERGY 0.5000 0.2500 0.3333 4 ASSESSMENT 0.0000 0.0000 0.0000 4 CC 0.0000 0.0000 0.0000 4 DIAGNOSIS 0.0000 0.0000 0.0000 1 DISPOSITION 0.0000 0.0000 0.0000 2 EDCOURSE 0.0000 0.0000 0.0000 3 EXAM 0.0000 0.0000 0.0000 1 FAM/SOCHX 0.8462 1.0000 0.9167 22 GENHX 0.7600 0.9500 0.8444 20 GYNHX 0.0000 0.0000 0.0000 1 IMAGING 0.0000 0.0000 0.0000 1 IMMUNIZATIONS 0.0000 0.0000 0.0000 1 LABS 0.0000 0.0000 0.0000 1 MEDICATIONS 0.5714 0.5714 0.5714 7 OTHER_HISTORY 0.0000 0.0000 0.0000 1 PASTMEDICALHX 0.3636 1.0000 0.5333 4 PASTSURGICAL 0.8000 1.0000 0.8889 8 PLAN 0.0000 0.0000 0.0000 3 PROCEDURES 0.0000 0.0000 0.0000 1 ROS 1.0000 0.0909 0.1667 11 accuracy 0.5900 100 macro avg 0.2421 0.2431 0.2127 100 weighted avg 0.5867 0.5900 0.5347 100
4,0.608300,0.526030,precision recall f1-score support ALLERGY 1.0000 0.2500 0.4000 4 ASSESSMENT 0.0000 0.0000 0.0000 4 CC 0.0000 0.0000 0.0000 4 DIAGNOSIS 0.0000 0.0000 0.0000 1 DISPOSITION 0.0000 0.0000 0.0000 2 EDCOURSE 0.0000 0.0000 0.0000 3 EXAM 0.0000 0.0000 0.0000 1 FAM/SOCHX 0.8148 1.0000 0.8980 22 GENHX 0.6552 0.9500 0.7755 20 GYNHX 0.0000 0.0000 0.0000 1 IMAGING 0.0000 0.0000 0.0000 1 IMMUNIZATIONS 0.0000 0.0000 0.0000 1 LABS 0.0000 0.0000 0.0000 1 MEDICATIONS 0.5714 0.5714 0.5714 7 OTHER_HISTORY 0.0000 0.0000 0.0000 1 PASTMEDICALHX 0.3636 1.0000 0.5333 4 PASTSURGICAL 0.7273 1.0000 0.8421 8 PLAN 0.0000 0.0000 0.0000 3 PROCEDURES 0.0000 0.0000 0.0000 1 ROS 1.0000 0.1818 0.3077 11 What is the DISPOSITION? 0.0000 0.0000 0.0000 0 accuracy 0.6000 100 macro avg 0.2444 0.2359 0.2061 100 weighted avg 0.5730 0.6000 0.5312 100
5,0.608300,0.416765,precision recall f1-score support ALLERGY 0.6667 0.5000 0.5714 4 ASSESSMENT 0.0000 0.0000 0.0000 4 CC 0.0000 0.0000 0.0000 4 DIAGNOSIS 0.0000 0.0000 0.0000 1 DISPOSITION 0.0000 0.0000 0.0000 2 EDCOURSE 0.0000 0.0000 0.0000 3 EXAM 0.5000 1.0000 0.6667 1 FAM/SOCHX 0.8800 1.0000 0.9362 22 GENHX 0.7600 0.9500 0.8444 20 GYNHX 0.0000 0.0000 0.0000 1 How are you feeling? 0.0000 0.0000 0.0000 0 IMAGING 0.0000 0.0000 0.0000 1 IMMUNIZATIONS 0.0000 0.0000 0.0000 1 LABS 0.0000 0.0000 0.0000 1 MEDICATION

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarnin

events.out.tfevents.1705276494.6844ab8511f4.290.0:   0%|          | 0.00/12.0k [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [8]:
# test
inputs = [
    "classify this dialogue: Doctor: So are you taking any medications at the moment?\nPatient: yes, I take tirosint every morning.\nDoctor: what dosage?\nPatient:125mg."
]
# inputs = ['classify this dialogue: Doctor: \nPatient: \nDoctor: ']

inputs = tokenizer(inputs, return_tensors="pt")
outputs = model.generate(**inputs)
category = tokenizer.decode(outputs[0])
category

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'<pad> MEDICATIONS</s>'